In [1]:
# =========================================================
# 1. Load FMCS raw data
# 2. Clean FMCS datasets
# 3. Generate Key Dependent Varialbes
# 3. Merge CBP data (number of firms & employees per state per year)
# 4. Merge union representation data
# =========================================================

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

# ---------- Paths ----------
# Define project directory
project_dir = Path("C:/Users/20234503/Desktop/Research/Strikes, Temperature, and Heat Safety Laws")
# Define raw CSV path
raw_csv = project_dir / "Raw Data" / "work_stoppages.csv"


# ---------- Load ----------
# Specify encoding/dtypes/dates for deterministic loads across machines.
FMCS = pd.read_csv(
    raw_csv,
    encoding="utf-8",
)

# ---------- Clean 'City, State' ----------
# Goal: split combined place field into tidy 'City' and 'State' for merges.
# 1) normalise trailing punctuation/whitespace (e.g., "Seattle, WA," -> "Seattle, WA")
FMCS["City, State"] = (
    FMCS["City, State"]
    .astype("string")
    .str.replace(r",$", "", regex=True)  # drop trailing comma if present
    .str.strip()
)

# 2) split on the LAST comma only (handles cities containing commas upstream, if any)
city_state = FMCS["City, State"].str.rsplit(",", n=1, expand=True)

# 3) assign to new columns with whitespace trimmed
FMCS["City"] = city_state[0].str.strip()     # everything before last comma
FMCS["State"] = city_state[1].str.strip()    # text after last comma

# 4) standardise missing/blank states to NA for consistent filtering/joins
FMCS["State"] = (
    FMCS["State"]
    .astype("string")
    .str.strip()
    .replace(r"^\s*$", pd.NA, regex=True)
)

# ---------- Diagnostics (optional but useful for replication logs) ----------
# Rows where state is missing after the split (likely malformed 'City, State')
bad_rows = FMCS[FMCS["State"].isna()]

# If desired in strict pipelines, fail fast when structure isn't as expected:
# assert bad_rows.empty, "Some rows lack a parseable state; inspect 'bad_rows'."

# Example: inspect a minimal subset when debugging
# print(bad_rows[["Employer", "City, State"]].head(10))

# ---------- Crosswalk: FMCS 'Industry' -> NAICS2 ----------
# Assumptions:
# - FMCS Industry values map to broad NAICS 2-digit sectors (some as ranges, e.g. "48-49").
# - Unrecognised/unspecified industries remain NA (documented in logs).
# - We standardise text before mapping to reduce mismatches (strip and normalise spaces).

# 0) normalise 'Industry' text to reduce key mismatches during mapping
FMCS["Industry"] = (
    FMCS["Industry"]
    .astype("string")
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)   # collapse multiple spaces
)

# 1) dictionary crosswalk from FMCS labels to NAICS 2-digit (as strings)
crosswalk = {
    "Manufacturing": "31-33",
    "Information": "51",
    "Construction": "23",
    "Health Care and Social Assistance": "62",
    "Educational Services": "61",
    "Transportation and Warehousing": "48-49",
    "Utilities": "22",
    "Retail Trade": "44-45",
    "Wholesale Trade": "42",
    "Real Estate and Rental & Leasing": "53",
    "Professional, Scientific &Tech Serv": "54",
    "Finance and Insurance": "52",
    "Accommodation and Food Services": "72",
    "Arts, Entertainment and Recreation": "71",
    "Mining, Quarrying and Oil & Gas Extraction": "21",
    "Mining and Oil & Gas Extraction": "21",
    "Support Serv. & Waste Management": "56",
    "Personal Serv & Private Organizations": "81",

    # Government (aggregate to 92)
    "Federal Government": "92",
    "State Government": "92",
    "Local Government": "92",
    "County Government": "92",
    "Federal, State or Local Government": "92",

    # Explicit unknowns
    "Not Specified": pd.NA,
    "Not Provided": pd.NA,
    "nan": pd.NA,
}

FMCS["NAICS2"] = FMCS["Industry"].map(crosswalk)

# Optional diagnostic: count unmapped industries (helps maintain the crosswalk over time)
# unmapped = FMCS.loc[FMCS["NAICS2"].isna(), "Industry"].value_counts(dropna=False)
# print("Unmapped industry labels:\n", unmapped.head(20))

# ---------- Creation of State-Year Panel of Labour Strikes by Indoor & Outdoor Sector ----------
# Objectives:
# - Generate state-year strike incidence
#   (if bargaining power increases, we should observe more unrest).
# - Calculate average dispute length
#   (captures how long disputes typically take to resolve).
# - Classify disputes as short vs. long
#   (literature suggests shorter disputes are more likely to be successful;
#    cutoff to be defined explicitly — e.g., 7, 14, or 30 days).
# - Measure severity of disputes
#   (defined as number of workers idle × duration in days; normalisation by state size may be needed).
# - Contract content
#   (text evidence from collective bargaining agreements; data not yet available).
# - Enforcement/pressure channels
#   (e.g., OSHA / Cal-OSHA inspections & citations, NLRB unfair labour practice charges; data not yet available).

# ---------- Study Period -----------
FMCS["Start Date"] = pd.to_datetime(FMCS["Start Date"], errors="coerce") # Convert to datetime (keeps invalids as NaT)
FMCS["End Date"]   = pd.to_datetime(FMCS["End Date"], errors="coerce")
FMCS["Year"] = FMCS["Start Date"].dt.year # Extract year from Start Date
FMCS = FMCS[(FMCS["Year"] >= 2000) & (FMCS["Year"] <= 2010)] # Restrict sample to 2000–2010

# ---------- Incidence -----------
FMCS = FMCS.reset_index(drop=True)
FMCS["DisputeID"] = (
    FMCS["State"].astype(str) + "_" +
    FMCS["Year"].astype(str) + "_" +
    FMCS.index.astype(str)
)
# ---------- Duration of Disputes ----------
# Duration in days (End - Start)
FMCS["Dispute_Duration_Days"] = (FMCS["End Date"] - FMCS["Start Date"]).dt.days
FMCS.loc[FMCS["Dispute_Duration_Days"] == 0, "Dispute_Duration_Days"] = 1
# Clean impossible values
FMCS.loc[FMCS["Dispute_Duration_Days"] < 0, "Dispute_Duration_Days"] = pd.NA

# Short vs. Long Disputes 
SHORT_CUTOFF = 6
FMCS["Short_Dispute"] = np.where(
    FMCS["Dispute_Duration_Days"].isna(),
    np.nan,
    FMCS["Dispute_Duration_Days"].le(SHORT_CUTOFF)
)

# ---------- Dispute Severity ----------
FMCS["Severity"] = FMCS["Dispute_Duration_Days"]*FMCS["# Idled"]

# Aggregate to state × NAICS2 × year
State_Year_FMCS = (
    FMCS.groupby(["State", "NAICS2", "Year"], dropna=False)
        .agg(
            Count=("DisputeID", "count"),           # number of disputes
            Avg_Length=("Dispute_Duration_Days", "mean"),  # average duration
            Severity=("Severity", "sum"),           # total idle days × workers
            Short_Share=("Short_Dispute", "mean")   # proportion short disputes
        )
        .reset_index()
)
# ---------- Ensure all state-sector-year combinations are present ----------
# This ensures that states/years with no disputes are still represented with zeros.
# This is crucial for panel data analysis to avoid bias from missing data.
# Create a "date_year" timestamp (Jan 1 of each Year)
State_Year_FMCS["date_year"] = pd.to_datetime(
    State_Year_FMCS["Year"].astype(int).astype(str), format="%Y"
)

# Build a range covering ALL years in your data (first day of each year)
all_years = pd.date_range(
    start=f"{int(State_Year_FMCS['Year'].min())}-01-01",
    end=f"{int(State_Year_FMCS['Year'].max())}-01-01",
    freq="YS"   # Year Start
)

# Select relevant columns (adjust based on available columns)
state_df  = (State_Year_FMCS[["State"]]
             .drop_duplicates()
             .dropna()
             .sort_values("State")
             .reset_index(drop=True))

sector_df = (State_Year_FMCS[["NAICS2"]]
             .drop_duplicates()
             .dropna()
             .sort_values("NAICS2")
             .reset_index(drop=True))


#Merging with temporal data
all_combinations = pd.MultiIndex.from_product([state_df['State'], sector_df['NAICS2'], all_years], names=['State', 'NAICS2','date_year']).to_frame(index=False)
State_Year_FMCS= pd.merge(State_Year_FMCS, all_combinations, on=['State', 'NAICS2','date_year'], how='right') # how='right' creates a dataset of all counties
# Or store as a yearly Period instead of a timestamp
State_Year_FMCS["Year"] = State_Year_FMCS["date_year"].dt.year # Extract year from Start Date
State_Year_FMCS["year_period"] = pd.PeriodIndex(
    State_Year_FMCS["Year"].astype(int), freq="Y"
)

# ---------- Classify Sector Type: Indoor vs Outdoor ----------
# Rule of thumb:
# Outdoor-heavy sectors include natural resources, extraction, utilities/field work, construction,
# transport/warehousing (field roles), waste mgmt, and government field operations.
# NOTE: This is a coarse classification for heat exposure analysis, not a job-level exposure measure.

outdoor_naics = {
    "11",      # Agriculture, forestry, fishing and hunting
    "21",      # Mining, quarrying, oil & gas extraction
    "22",      # Utilities (field crews)
    "23",      # Construction
    "48-49",   # Transportation and warehousing
    "56",      # Admin/support & waste management (incl. waste collection)
    "92",      # Public administration (field services; coarse assumption)
}

def classify_sector(naics2: pd.Series) -> str | None:
    if pd.isna(naics2):
        return None
    return "Outdoor" if naics2 in outdoor_naics else "Indoor"

State_Year_FMCS["Sector_Type"] = State_Year_FMCS["NAICS2"].apply(classify_sector)

State_Year_FMCS["outdoor"] = State_Year_FMCS["Sector_Type"].astype(str).str.lower().eq("outdoor").astype(int)

cols = ["Count", "Avg_Length", "Severity", "Short_Share"]
present = [c for c in cols if c in State_Year_FMCS.columns]
State_Year_FMCS[present] = State_Year_FMCS[present].apply(pd.to_numeric, errors="coerce").fillna(0)

# ---------- Merging of Data with CBP and Union Representation Data ----------
# State abbreviation to FIPS crosswalk
state_to_fips = {
    "AL": 1,  "AK": 2,  "AZ": 4,  "AR": 5,  "CA": 6,
    "CO": 8,  "CT": 9,  "DE": 10, "DC": 11, "FL": 12,
    "GA": 13, "HI": 15, "ID": 16, "IL": 17, "IN": 18,
    "IA": 19, "KS": 20, "KY": 21, "LA": 22, "ME": 23,
    "MD": 24, "MA": 25, "MI": 26, "MN": 27, "MS": 28,
    "MO": 29, "MT": 30, "NE": 31, "NV": 32, "NH": 33,
    "NJ": 34, "NM": 35, "NY": 36, "NC": 37, "ND": 38,
    "OH": 39, "OK": 40, "OR": 41, "PA": 42, "RI": 44,
    "SC": 45, "SD": 46, "TN": 47, "TX": 48, "UT": 49,
    "VT": 50, "VA": 51, "WA": 53, "WV": 54, "WI": 55,
    "WY": 56,
    # Territories
    "PR": 72, "VI": 78,
    # Placeholder for missing/unknown
    "Unknown": None
}

State_Year_FMCS["FIPSSTATE"] = (
    State_Year_FMCS["State"].map(state_to_fips).astype("Int64")
)  # nullable integer dtype

from pathlib import Path
import re
import pandas as pd

# ---------- Setup ----------
CBP_dir = project_dir / "Raw Data" / "CBP"

all_dfs = []
for fp in sorted(CBP_dir.glob("Cbp*st.txt")):
    m = re.fullmatch(r"Cbp(\d{2})st\.txt", fp.name, flags=re.IGNORECASE)
    if not m:
        continue  # skip unexpected files

    year = 2000 + int(m.group(1))

    df = pd.read_csv(fp, encoding="utf-8")

    # Apply filter
    if year == 2010:
        mask = (df["naics"] == "------") & (df["lfo"] == "-")
    else:
        mask = df["naics"] == "------"

    df = df.loc[mask, ["fipstate", "emp", "est"]].copy()
    df["year"] = year
    all_dfs.append(df)

# Combine
CBP_State_Tot = pd.concat(all_dfs, ignore_index=True)[["year", "fipstate", "emp", "est"]]

#Merging CBP_State_Tot with State_Year_FMCS
# Standardize CBP column names to match the base df
CBP_aligned = (
    CBP_State_Tot
      .rename(columns={"fipstate": "FIPSSTATE", "year": "Year"})  # keys
      .rename(columns={"emp": "cbp_emp", "est": "cbp_est"})       # payload (optional but recommended)
      .copy()
)

# Make sure dtypes line up
State_Year_FMCS["FIPSSTATE"] = State_Year_FMCS["FIPSSTATE"].astype("Int64")
State_Year_FMCS["Year"]      = State_Year_FMCS["Year"].astype("Int64")

CBP_aligned["FIPSSTATE"] = CBP_aligned["FIPSSTATE"].astype("Int64")
CBP_aligned["Year"]      = CBP_aligned["Year"].astype("Int64")

# Merge on the matching names
State_Year_FMCS_Rate = State_Year_FMCS.merge(
    CBP_aligned[["FIPSSTATE", "Year", "cbp_emp", "cbp_est"]],
    on=["FIPSSTATE", "Year"],
    how="left"
)

#Creating Rates of Dependent Variable
State_Year_FMCS_Rate['Count'] = (State_Year_FMCS_Rate['Count']/State_Year_FMCS_Rate['cbp_emp'])*1000000
State_Year_FMCS_Rate['Severity'] = (State_Year_FMCS_Rate['Severity']/State_Year_FMCS_Rate['cbp_emp'])*1000000
State_Year_FMCS_Rate['Severity'] = (State_Year_FMCS_Rate['Severity']/State_Year_FMCS_Rate['cbp_emp'])*1000000

######## Export ######
State_Year_FMCS_Rate.to_csv("C:\\Users\\20234503\\Desktop\\Research\\Strikes, Temperature, and Heat Safety Laws\\Project\Data\\"+"State_Year_FMCS_Rate.csv", index=False)
State_Year_FMCS.to_csv("C:\\Users\\20234503\\Desktop\\Research\\Strikes, Temperature, and Heat Safety Laws\\Project\Data\\"+"State_Year_FMCS.csv", index=False)

In [13]:
# ---------- Creation of State-Month Panel of Labour Strikes by Indoor & Outdoor Sector ----------
# Objectives:
# - Generate state-year strike incidence
#   (if bargaining power increases, we should observe more unrest).
# - Calculate average dispute length
#   (captures how long disputes typically take to resolve).
# - Classify disputes as short vs. long
#   (literature suggests shorter disputes are more likely to be successful;
#    cutoff to be defined explicitly — e.g., 7, 14, or 30 days).
# - Measure severity of disputes
#   (defined as number of workers idle × duration in days; normalisation by state size may be needed).
# - Contract content
#   (text evidence from collective bargaining agreements; data not yet available).
# - Enforcement/pressure channels
#   (e.g., OSHA / Cal-OSHA inspections & citations, NLRB unfair labour practice charges; data not yet available).

# ---------- Study Period -----------
FMCS["Start Date"] = pd.to_datetime(FMCS["Start Date"], errors="coerce") # Convert to datetime (keeps invalids as NaT)
FMCS["End Date"]   = pd.to_datetime(FMCS["End Date"], errors="coerce")
FMCS["Month"] = FMCS["Start Date"].dt.month # Extract year from Start Date
FMCS["Year"] = FMCS["Start Date"].dt.year # Extract year from Start Date
FMCS = FMCS[(FMCS["Year"] >= 2000) & (FMCS["Year"] <= 2010)] # Restrict sample to 2000–2010

# ---------- Incidence -----------
FMCS = FMCS.reset_index(drop=True)
FMCS["DisputeID"] = (
    FMCS["State"].astype(str) + "_" +
    FMCS["Year"].astype(str) + "_" +
    FMCS.index.astype(str)
)
# ---------- Duration of Disputes ----------
# Duration in days (End - Start)
FMCS["Dispute_Duration_Days"] = (FMCS["End Date"] - FMCS["Start Date"]).dt.days
FMCS.loc[FMCS["Dispute_Duration_Days"] == 0, "Dispute_Duration_Days"] = 1
# Clean impossible values
FMCS.loc[FMCS["Dispute_Duration_Days"] < 0, "Dispute_Duration_Days"] = pd.NA

# Short vs. Long Disputes 
SHORT_CUTOFF = 6
FMCS["Short_Dispute"] = np.where(
    FMCS["Dispute_Duration_Days"].isna(),
    np.nan,
    FMCS["Dispute_Duration_Days"].le(SHORT_CUTOFF)
)

# ---------- Dispute Severity ----------
FMCS["Severity"] = FMCS["Dispute_Duration_Days"]*FMCS["# Idled"]

# Aggregate to state × NAICS2 × year
State_Month_FMCS = (
    FMCS.groupby(["State", "NAICS2", "Month", "Year"], dropna=False)
        .agg(
            Count=("DisputeID", "count"),           # number of disputes
            Avg_Length=("Dispute_Duration_Days", "mean"),  # average duration
            Severity=("Severity", "sum"),           # total idle days × workers
            Short_Share=("Short_Dispute", "mean")   # proportion short disputes
        )
        .reset_index()
)

# ---------- Ensure all state-sector-year combinations are present ----------
# This ensures that states/years with no disputes are still represented with zeros.
# This is crucial for panel data analysis to avoid bias from missing data.
# Create a "date_year" timestamp (Jan 1 of each Year)
State_Month_FMCS['date_month'] = pd.to_datetime(
    State_Month_FMCS['Year'].astype(str) + '-' + State_Month_FMCS['Month'].astype(str) + '-01'
)

# Build a range covering ALL years in your data (first day of each year)
all_months = pd.date_range(
    start=State_Month_FMCS['date_month'].min(),
    end=State_Month_FMCS['date_month'].max(),
    freq='MS'  # Month Start
)

# Select relevant columns (adjust based on available columns)
state_df  = (State_Month_FMCS[["State"]]
             .drop_duplicates()
             .dropna()
             .sort_values("State")
             .reset_index(drop=True))

sector_df = (State_Month_FMCS[["NAICS2"]]
             .drop_duplicates()
             .dropna()
             .sort_values("NAICS2")
             .reset_index(drop=True))


#Merging with temporal data
all_combinations = pd.MultiIndex.from_product([state_df['State'], sector_df['NAICS2'], all_months], names=['State', 'NAICS2','date_month']).to_frame(index=False)
State_Month_FMCS= pd.merge(State_Month_FMCS, all_combinations, on=['State', 'NAICS2','date_month'], how='right') # how='right' creates a dataset of all counties

State_Month_FMCS["Year"] = State_Month_FMCS["date_month"].dt.year # Extract year from Start Date
State_Month_FMCS["Month"] = State_Month_FMCS["date_month"].dt.month # Extract year from Start Date
# ---------- Classify Sector Type: Indoor vs Outdoor ----------
# Rule of thumb:
# Outdoor-heavy sectors include natural resources, extraction, utilities/field work, construction,
# transport/warehousing (field roles), waste mgmt, and government field operations.
# NOTE: This is a coarse classification for heat exposure analysis, not a job-level exposure measure.

outdoor_naics = {
    "11",      # Agriculture, forestry, fishing and hunting
    "21",      # Mining, quarrying, oil & gas extraction
    "22",      # Utilities (field crews)
    "23",      # Construction
    "48-49",   # Transportation and warehousing
    "56",      # Admin/support & waste management (incl. waste collection)
    "92",      # Public administration (field services; coarse assumption)
}

def classify_sector(naics2: pd.Series) -> str | None:
    if pd.isna(naics2):
        return None
    return "Outdoor" if naics2 in outdoor_naics else "Indoor"

State_Month_FMCS["Sector_Type"] = State_Month_FMCS["NAICS2"].apply(classify_sector)

State_Month_FMCS["outdoor"] = State_Month_FMCS["Sector_Type"].astype(str).str.lower().eq("outdoor").astype(int)

cols = ["Count", "Avg_Length", "Severity", "Short_Share"]
present = [c for c in cols if c in State_Year_FMCS.columns]
State_Month_FMCS[present] = State_Month_FMCS[present].apply(pd.to_numeric, errors="coerce").fillna(0)
# ---------- Merging of Data with CBP and Union Representation Data ----------
# State abbreviation to FIPS crosswalk
state_to_fips = {
    "AL": 1,  "AK": 2,  "AZ": 4,  "AR": 5,  "CA": 6,
    "CO": 8,  "CT": 9,  "DE": 10, "DC": 11, "FL": 12,
    "GA": 13, "HI": 15, "ID": 16, "IL": 17, "IN": 18,
    "IA": 19, "KS": 20, "KY": 21, "LA": 22, "ME": 23,
    "MD": 24, "MA": 25, "MI": 26, "MN": 27, "MS": 28,
    "MO": 29, "MT": 30, "NE": 31, "NV": 32, "NH": 33,
    "NJ": 34, "NM": 35, "NY": 36, "NC": 37, "ND": 38,
    "OH": 39, "OK": 40, "OR": 41, "PA": 42, "RI": 44,
    "SC": 45, "SD": 46, "TN": 47, "TX": 48, "UT": 49,
    "VT": 50, "VA": 51, "WA": 53, "WV": 54, "WI": 55,
    "WY": 56,
    # Territories
    "PR": 72, "VI": 78,
    # Placeholder for missing/unknown
    "Unknown": None
}

State_Month_FMCS["FIPSSTATE"] = (
    State_Month_FMCS["State"].map(state_to_fips).astype("Int64")
)  # nullable integer dtype

from pathlib import Path
import re
import pandas as pd

# ---------- Setup ----------
CBP_dir = project_dir / "Raw Data" / "CBP"

all_dfs = []
for fp in sorted(CBP_dir.glob("Cbp*st.txt")):
    m = re.fullmatch(r"Cbp(\d{2})st\.txt", fp.name, flags=re.IGNORECASE)
    if not m:
        continue  # skip unexpected files

    year = 2000 + int(m.group(1))

    df = pd.read_csv(fp, encoding="utf-8")

    # Apply filter
    if year == 2010:
        mask = (df["naics"] == "------") & (df["lfo"] == "-")
    else:
        mask = df["naics"] == "------"

    df = df.loc[mask, ["fipstate", "emp", "est"]].copy()
    df["year"] = year
    all_dfs.append(df)

# Combine
CBP_State_Tot = pd.concat(all_dfs, ignore_index=True)[["year", "fipstate", "emp", "est"]]

#Merging CBP_State_Tot with State_Year_FMCS
# Standardize CBP column names to match the base df
CBP_aligned = (
    CBP_State_Tot
      .rename(columns={"fipstate": "FIPSSTATE", "year": "Year"})  # keys
      .rename(columns={"emp": "cbp_emp", "est": "cbp_est"})       # payload (optional but recommended)
      .copy()
)

# Make sure dtypes line up
State_Month_FMCS["FIPSSTATE"] = State_Month_FMCS["FIPSSTATE"].astype("Int64")
State_Month_FMCS["Year"]      = State_Month_FMCS["Year"].astype("Int64")

CBP_aligned["FIPSSTATE"] = CBP_aligned["FIPSSTATE"].astype("Int64")
CBP_aligned["Year"]      = CBP_aligned["Year"].astype("Int64")

# Merge on the matching names
State_Month_FMCS_Rate = State_Month_FMCS.merge(
    CBP_aligned[["FIPSSTATE", "Year", "cbp_emp", "cbp_est"]],
    on=["FIPSSTATE", "Year"],
    how="left"
)

#Creating Rates of Dependent Variable
State_Month_FMCS_Rate['Count'] = (State_Month_FMCS_Rate['Count']/State_Month_FMCS_Rate['cbp_emp'])*1000000
State_Month_FMCS_Rate['Severity'] = (State_Month_FMCS_Rate['Severity']/State_Month_FMCS_Rate['cbp_emp'])*1000000
State_Month_FMCS_Rate['Severity'] = (State_Month_FMCS_Rate['Severity']/State_Month_FMCS_Rate['cbp_emp'])*1000000

######## Export ######
State_Month_FMCS_Rate.to_csv("C:\\Users\\20234503\\Desktop\\Research\\Strikes, Temperature, and Heat Safety Laws\\Project\Data\\"+"State_Month_FMCS_Rate.csv", index=False)
State_Month_FMCS.to_csv("C:\\Users\\20234503\\Desktop\\Research\\Strikes, Temperature, and Heat Safety Laws\\Project\Data\\"+"State_Month_FMCS.csv", index=False)